In [ ]:
### Ligand file creation - how exactly do we generate the SDF files for novel lignands?

import os
import subprocess
from SBLMDCOVDOCK.SBLSettings import GROMACS_Settings
import requests
settings = GROMACS_Settings()

In [ ]:
# use: https://ambermd.org/tutorials/basic/tutorial5/index.php
# only partially works with GAFF - doesnt generate charges.

resname = "KCX"

# download from PDB
# 
link = f"https://files.rcsb.org/ligands/download/{resname}.cif"

# download the file to KCX_dir

KCX_dir = os.path.join(settings.structures_input, resname)

# Check if the directory exists, if not, create it
if not os.path.exists(KCX_dir):
    os.makedirs(KCX_dir)

# Download the CIF file and save it to the target directory
cif_file_path = os.path.join(KCX_dir, f"{resname}.cif")
response = requests.get(link)

if response.status_code == 200:
    with open(cif_file_path, "wb") as f:
        f.write(response.content)
    print(f"CIF file downloaded and saved to {cif_file_path}")
else:
    print("Failed to download the CIF file.")

In [ ]:


print(KCX_dir)

KCX_cif_path = os.path.join(KCX_dir, resname+'.cif')

antechamber_command = [
    'antechamber',
    '-i', KCX_cif_path,
    '-fi', 'ccif',
    "-bk", resname, # component block for ccif
    '-o', KCX_cif_path.replace('.cif', '.ac'),
    '-fo', 'ac',
    '-c', 'bcc', # charge method
    '-at', 'amber', # ff atom types
    '-s', '2', # verbose
    '-nc', '0'

]

print(" ".join(antechamber_command))

subprocess.run(antechamber_command, check=True)

In [ ]:
KCX_mc = """
HEAD_NAME N
TAIL_NAME C
MAIN_CHAIN CA
MAIN_CHAIN CB
MAIN_CHAIN CG
MAIN_CHAIN CD
MAIN_CHAIN CE
MAIN_CHAIN NZ
MAIN_CHAIN CX
MAIN_CHAIN OQ1
MAIN_CHAIN OQ2
OMIT_NAME HN2
OMIT_NAME OXT
OMIT_NAME HXT
OMIT_NAME HQ2
PRE_HEAD_TYPE C
POST_TAIL_TYPE N
CHARGE -1.0
"""

KCX_mc = """
HEAD_NAME N
TAIL_NAME C
MAIN_CHAIN CA
OMIT_NAME HN2
OMIT_NAME OXT
OMIT_NAME HXT
OMIT_NAME HQ2
PRE_HEAD_TYPE C
POST_TAIL_TYPE N
CHARGE -1.0
"""

path = os.path.join(KCX_dir, resname+'.mc')
print(path)
with open(path, 'w') as f:
    f.write(KCX_mc)



In [ ]:

prepgen_command = [
    "prepgen",
    "-i", resname+'.ac',
    "-o", resname+'.prepin',
    "-m", resname+'.mc',
    "-rn", resname,
]
    
print(" ".join(prepgen_command))
subprocess.run(prepgen_command, check=True, cwd=KCX_dir)

In [ ]:
# find
!open $AMBERHOME

In [ ]:
# adapt parmchk2 -i cro.prepin -f prepi -o frcmod.cro -a Y -p $AMBERHOME/dat/leap/parm/parm10.dat

ff_dat = "parm19.dat"

parmchk_command = [
    "parmchk2",
    "-i", resname + ".prepin",
    "-f", "prepi",
    "-o", "frcmod." + resname,
    "-a", "Y",
    "-p", os.environ["AMBERHOME"]+"/dat/leap/parm/"+ ff_dat
]

print(" ".join(parmchk_command))
subprocess.run(parmchk_command, check=True, cwd=KCX_dir)

In [ ]:
# strip out ATTN lines

# ! grep -v "ATTN" {"frcmod."+resname} > {"frcmod1."+resname} # Strip out ATTN lines

grep_command = [
    "grep", "-v", "ATTN",
    os.path.join(KCX_dir, "frcmod."+resname),
    ">", os.path.join(KCX_dir, "frcmod1."+resname)
]

# Run grep command
print(" ".join(grep_command))
! {" ".join(grep_command)}
# subprocess.run(grep_command, check=True)

In [ ]:
# GAFF parameters for ligands
# adapt parmchk2 -i cro.prepin -f prepi -o frcmod.cro 


parmchk_command = [
    "parmchk2",
    "-i", resname + ".prepin",
    "-f", "prepi",
    "-o", "frcmod2." + resname,
    # "-a", "Y",
    # "-p", os.environ["AMBERHOME"]+"/dat/leap/parm/"+ ff_dat
]

print(" ".join(parmchk_command))
subprocess.run(parmchk_command, check=True, cwd=KCX_dir)

In [ ]:

leap_file = f"""source leaprc.protein.ff19SB
set default PBRadii mbondi3
loadAmberPrep {resname}.prepin
loadAmberParams frcmod2.{resname}
loadAmberParams frcmod1.{resname}
x = loadPDB gfp.pdb
saveAmberParm x gfp.parm7 gfp.rst7
quit
"""